In [1]:
import torch
import torch.nn as nn
import cv2 # pip install opencv-python


In [ ]:
def CAM(net, conv_filter, layer, org_shape, cls):
    act = {}
    def get_activation(name):
        def hook(net, inp, out):
            act[name] = out.detach().numpy()
        return hook
    layer.register_forward_hook(get_activation('target')) # forward hook을 등록
    linear = net.classifier[6].weight[cls].detach().numpy()[:, np.newaxis, np.newaxis] # w1, w2, ..., wn -> weight
    _ = net(conv_filter) # forward hook을 실행시킴
    activation = act['target'][0] # spatial한 conv filter 가져오는 코드
    activation = activation.sum(axis=0)[np.newaxis, ...]
    activation = activation * linear 
    activation = activation.mean(axis=0)
    cam = cv2.resize(activation.squeeze(), org_shape, cv2.INTER_LINEAR)
    return cam
